In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
from sparsepixels.utils import *

#run_batch(which="inclusive", overwrite_outputs=True, store_all=False, sig_hit_threshold=50)
#run_batch(which="inclusive", overwrite_outputs=True, store_all=True)
#run_batch(which="nue", overwrite_outputs=True, store_all=True)

In [ ]:
#plot_plane_sample("datasets/neutrino/data_inclusive_plane2_slim_500.h5", idx=0, p='raw',   jet_vmin=0, jet_vmax=100, cmap='gist_ncar')
#plot_plane_sample("datasets/neutrino/data_inclusive_plane2_slim_500.h5", idx=0, p='sig', point_size=3)
#plot_plane_sample("datasets/neutrino/data_inclusive_plane2_slim_500.h5", idx=0, p='bkg', point_size=3)
#plot_plane_sample("datasets/neutrino/data_inclusive_plane2_slim_500.h5", idx=0, p='sigbkg', point_size=3)

#plot_plane_sample("datasets/neutrino/data_nue_plane2_slim_500.h5", idx=1, p='raw',   jet_vmin=0, jet_vmax=100, cmap='gist_ncar')
#plot_plane_sample("datasets/neutrino/data_nue_plane2_slim_500.h5", idx=1, p='sig', point_size=3)
#plot_plane_sample("datasets/neutrino/data_nue_plane2_slim_500.h5", idx=1, p='bkg', point_size=3)
#plot_plane_sample("datasets/neutrino/data_nue_plane2_slim_500.h5", idx=1, p='sigbkg', point_size=3)

for i in range(10):
    plot_plane_sample("datasets/neutrino/data_inclusive_plane2_slim_500.h5", idx=i, p='raw',   jet_vmin=0, jet_vmax=100, cmap='gist_ncar')
    plot_plane_sample("datasets/neutrino/data_inclusive_plane2_slim_500.h5", idx=i, p='sigbkg', point_size=3)
    #plot_plane_sample("datasets/neutrino/data_nue_plane2_slim_500.h5", idx=i, p='raw',   jet_vmin=0, jet_vmax=100, cmap='gist_ncar')
    #plot_plane_sample("datasets/neutrino/data_nue_plane2_slim_500.h5", idx=i, p='sigbkg', point_size=3)

In [ ]:
run_window_demo(
    "datasets/neutrino/data_nue_plane2_slim_500.h5",
    first_N=5, win_t=256, win_w=512, num_bkg=5,
    prefer_zero_signal_bkg=True,
    cmap='gist_ncar', vmin=0, vmax=100,
    rng_seed=123
)